In [1]:
import pandas as pd
import numpy as np

# Append Observed to Modeld (previously completed)

In [104]:
# Load the observed records
# 
# obs = pd.read_csv(r'R:\FastTrips\CHTS and OBS ft_output\OBS_ft_output_w_stops.csv')
# obs.fillna('',inplace=True)
# obs['person_id'] = obs['person_id'].apply(lambda x: x.split("_")[-1])
# obs['A_id_num'] = obs['A_id_num'].replace("",0)
# obs['A_id_num'] = obs['A_id_num'].astype('int')

# For testing, use a known list of trips, e.g., fast trips output
obs = pd.read_csv(r'J:\Projects\FasTrips\chts\output\CHTS_fasttrips_demand_v0.1_stochastic_iter2_nocap\chosenpaths_links.csv')
# Won't need to do this with real observed data
obs['route_id'].fillna("",inplace=True)
obs = obs[obs['iteration'] == obs['iteration'].max()]




# Load the model records
model = pd.read_csv(r'J:\Projects\FasTrips\chts\output\CHTS_fasttrips_demand_v0.1_stochastic_iter2_nocap\chosenpaths_links.csv')
model['route_id'].fillna("",inplace=True)
# FOR NOW, assume we look at only the last iteration of path sets ?
model = model[model['iteration'] == model['iteration'].max()]


# Load pathset links
pathset_links = pd.read_csv(r'J:\Projects\FasTrips\tableau\path-set\pathset_links.csv')
pathset_links['route_id'].fillna("",inplace=True)
pathset_links = pathset_links[pathset_links['iteration'] == pathset_links['iteration'].max()]

In [175]:
def produce_path_fields(df, group):
    '''
    '''
    # create "path_routes"
    df['path_routes'] = df['route_id'].apply(lambda x: x.strip())
    path_routes = pd.DataFrame(df.groupby(group)['path_routes'].apply(lambda x: "%s" % ' '.join(x).strip()))
    
    result_df = pd.DataFrame(index=path_routes.index)
    result_df['path_routes'] = path_routes
    
    # create "path_modes"
    df['path_modes'] = df['mode'].apply(lambda x: x.strip())
    result_df['path_modes'] = pd.DataFrame(df.groupby(group)['mode'].apply(lambda x: "%s" % ' '.join(x).strip()))

    # Create "path_components"
    df['path_components'] = df['A_id'].astype('str')+" "+df['mode']+" "+df['route_id']
    df['path_components'] = df['path_components'].apply(lambda x: x.strip())
    result_df['path_components'] = pd.DataFrame(df.groupby(['person_id','trip_list_id_num'])['path_components'].apply(lambda x: "%s" % ' '.join(x).strip()))
    
    # Return ID field from index
    result_df['id'] = result_df.index.get_level_values(0).values
    

    return result_df

# Produce joined fields from pathset link files 

In [176]:
# Create caluclated joined fields for the obsserved, modeled, and pathset links file
observed_path = produce_path_fields(obs, group=['person_id','trip_list_id_num'])
modeled_path = produce_path_fields(model, group=['person_id','trip_list_id_num'])
new_pathset = produce_path_fields(pathset_links, group=['person_id','trip_list_id_num','pathnum'])

In [113]:
# Join the observed and modeled fields
df = pd.merge(observed_path, modeled_path, on='id',suffixes=("_observed","_model"))

In [114]:
### Do this the dumb way for now
df['common_routes'] = np.zeros(len(df))
df['complete_match'] = np.zeros(len(df))
df['partial_match'] = np.zeros(len(df))
for i in xrange(len(df)):
    obs_list = filter(None,df['path_routes_observed'].iloc[i].split(" "))
    model_list = filter(None,df['path_routes_model'].iloc[i].split(" "))
    common = list(set(obs_list).intersection(model_list))
    df['common_routes'].iloc[i] = common
    
    # complete match field
    if abs(len(obs_list) - len(model_list)) == 0:
        df['complete_match'].iloc[i] = 1
    else:
        df['complete_match'].iloc[i] = 0
    
    if (len(common) > 0) & (len(obs_list) != len(model_list)):
        df['partial_match'].iloc[i] = 1
    else:
        df['partial_match'].iloc[i] = 0
    
    
#     print common

In [115]:
# SOMETHING LOOKS WRONG WITH THIS OUTPUT...

# % trips with matching or partial matching routes

In [116]:
# Now we find the percent of trips with matching routes or partial matching routes
df.mean()[['complete_match','partial_match']]
# Note, shuold be 100% when using the same data set, what's up??

complete_match    0.775401
partial_match     0.021390
dtype: float64

# Compare probability of observed path to pathset


In [117]:
# Load the model pathset paths
pathset_paths = pd.read_csv(r'J:\Projects\FasTrips\tableau\path-set\pathset_paths.csv')

## Check if path exists in the pathset

In [119]:
def check_path_existence(row, pathset_df, unique_id, lookup_field):
    """
    Compare field contents from chosen paths vs entire path set,
    Return 1 if chosen path exists in pathset, 0 otherwise
    """
    
    if len(pathset_df[(pathset_df[unique_id] == row[unique_id]) & \
                        (pathset_df[lookup_field] == row[lookup_field])]) > 0:
        return 1
    else:
        return 0

In [178]:
route_match = observed_path.apply(check_path_existence, pathset_df=new_pathset, unique_id='id', lookup_field='path_routes', axis=1)
mode_match = observed_path.apply(check_path_existence, pathset_df=new_pathset, unique_id='id', lookup_field='path_modes', axis=1)
total_match = observed_path.apply(check_path_existence, pathset_df=new_pathset, unique_id='id', lookup_field='path_components', axis=1)

In [179]:
df = pd.DataFrame([route_match,mode_match,total_match]).T
df['id'] = df.index.get_level_values(0).values
df.columns = ['route_match','mode_match','total_match','id']

In [160]:
# Join those results back into the observed path file
df = pd.merge(observed_path, df)

In [162]:
# Percent of the observed trips that are found in the pathset, based on matching routes, modes, and everything
df.mean()

route_match    1
mode_match     1
total_match    0
dtype: float64

# Get the probability of the corresponding path from the pathset

In [165]:
def get_path_attribute(row, pathset_df, unique_id, lookup_field, value_field):
    """
    For a chosen path, return parameter from path set.
    E.g., return path probability (value_field) for observed path, based on matching path description (lookup_field)
    """
    
    if len(pathset_paths[(pathset_df[unique_id] == row[unique_id]) & \
                        (pathset_df[lookup_field] == row[lookup_field])]) > 0:
        try:
            return pathset_paths[(pathset_df[unique_id] == row[unique_id]) &  \
                             (pathset_df[lookup_field] == row[lookup_field])][value_field].values[0]
        except:
            return 0
    else:
        return 0

In [199]:
new_pathset['pathnum'] = new_pathset.index.get_level_values(2)
new_pathset['trip_list_id_num'] = new_pathset.index.get_level_values(1)

In [203]:
# Join the probability to the new_pathset file
newest_pathset = pd.merge(new_pathset,pathset_paths,left_on=['id','trip_list_id_num','pathnum'], right_on=['person_id','trip_list_id_num','pathnum'])

In [204]:
route_based = observed_path.apply(get_path_attribute, pathset_df=newest_pathset, unique_id='id', lookup_field='path_routes', value_field='pf_probability', axis=1)

In [206]:
route_based.mean()

0.03736854441659149

In [10]:
# Convert fields to list
df['path_routes_observed_list'] = df['path_routes_observed'].apply(lambda x: filter(None,x.split(" ")))
df['path_routes_model_list'] = df['path_routes_model'].apply(lambda x: filter(None,x.split(" ")))

In [11]:
df['path_routes_observed_list'].isin(df['path_routes_model_list'].values())

TypeError: 'numpy.ndarray' object is not callable

In [ ]:
df['path_routes_model_list'].values()

In [ ]:
# Store the output in a new file


# Create "path routes"
obs['path_routes'] = obs['route_id'].apply(lambda x: x.strip())
obs_path_routes = pd.DataFrame(obs.groupby(['person_id','trip_list_id_num'])['path_routes'].apply(lambda x: "%s" % ' '.join(x).strip()))
# observed_paths.index = obs_path_routes.index
# observed_paths = pd.DataFrame(index=obs_path_routes.index)
# observed_paths['path_routes'] = obs_path_routes

In [ ]:
# produce observed_paths file



In [ ]:
# Store the output in a new file
observed_paths = pd.DataFrame()

# Create "path routes"
obs['path_routes'] = obs['route_id'].apply(lambda x: x.strip())
obs_path_routes = pd.DataFrame(obs.groupby(['person_id','trip_list_id_num'])['path_routes'].apply(lambda x: "%s" % ' '.join(x).strip()))
observed_paths.index = obs_path_routes.index
observed_paths['path_routes'] = obs_path_routes

In [ ]:
# Create "path_modes"
obs['path_modes'] = obs['mode'].apply(lambda x: x.strip())
obs_path_modes = pd.DataFrame(obs.groupby(['person_id','trip_list_id_num'])['mode'].apply(lambda x: "%s" % ' '.join(x).strip()))
observed_paths['path_modes'] = obs_path_modes

In [ ]:
# Create "path_components"
obs['path_components'] = obs['A_id_num'].astype('str')+" "+obs['mode']+" "+obs['route_id']
obs['path_components'] = obs['path_components'].apply(lambda x: x.strip())
obs_path_components = pd.DataFrame(obs.groupby(['person_id','trip_list_id_num'])['path_components'].apply(lambda x: "%s" % ' '.join(x).strip()))
observed_paths['path_components'] = obs_path_components

In [ ]:
# We need similar fields to compare against in the modeled data
